# Local Manga AI

This notebook launches a fully-local pipeline: **Qwen2.5 → SDXL → SAM → SDXL Inpaint → Page Composer** and exposes a public HTTPS URL via **Cloudflare Tunnel**.

Model folders:
- `models/qwen2.5/`
- `models/sdxl/`
- `models/sdxl_inpaint/`
- `models/sam/` (checkpoint file expected: `sam_vit_h_4b8939.pth`)


In [ ]:
import os

# Runtime settings (run this BEFORE loading any torch/diffusers models)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

# On L40S (48GB), it's safe to keep SAM on GPU. Override if needed: "cpu"
os.environ.setdefault("MANGA_AI_SAM_DEVICE", "cuda")

# Use Animagine XL 4.0 as the SDXL art model (Diffusers repo)
os.environ.setdefault("MANGA_AI_SDXL_REPO", "cagliostrolab/animagine-xl-4.0")

print("PYTORCH_CUDA_ALLOC_CONF=", os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))
print("MANGA_AI_SAM_DEVICE=", os.environ.get("MANGA_AI_SAM_DEVICE"))
print("MANGA_AI_SDXL_REPO=", os.environ.get("MANGA_AI_SDXL_REPO"))


In [ ]:
import sys
from pathlib import Path

# Detect paths
CWD = Path.cwd().resolve()
ROOT = CWD

# If we opened this notebook from inside ./manga_ai, ROOT is already the package folder.
# If we opened it from repo root, ROOT needs to be ./manga_ai.
if not ((ROOT / "models").exists() and (ROOT / "scripts").exists()):
    if (ROOT / "manga_ai" / "models").exists() and (ROOT / "manga_ai" / "scripts").exists():
        ROOT = (ROOT / "manga_ai").resolve()

REPO_ROOT = ROOT.parent

# Make sure we can `import manga_ai` regardless of where Jupyter was launched from.
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("CWD:", CWD)
print("ROOT (manga_ai folder):", ROOT)
print("REPO_ROOT:", REPO_ROOT)
print("Python:", sys.version)


In [ ]:
import numpy as np

print("NumPy:", np.__version__)

# If you see NumPy 2.x here, you may hit binary incompatibility crashes with matplotlib/scikit-learn.
# Fix by running the following in a cell, then RESTART the kernel:
# !pip install "numpy<2" --force-reinstall
# !pip install -r manga_ai/requirements.txt --force-reinstall


In [ ]:
import os
from manga_ai.scripts.model_downloader import ensure_models_downloaded
from manga_ai.scripts.pipeline import default_model_paths

paths = default_model_paths(ROOT)

# If using an alternative SDXL repo, keep its files in a dedicated folder to avoid mixing models.
os.environ.setdefault("MANGA_AI_SDXL_DIR", str(ROOT / "models" / "animagine_xl_4.0"))

print("Qwen dir:", paths.qwen_dir)
print("SDXL dir:", paths.sdxl_dir)
print("SDXL inpaint dir:", paths.sdxl_inpaint_dir)
print("SAM ckpt:", paths.sam_ckpt)

# This will download models into the folders above if they are missing.
# For gated HuggingFace models, set: MANGA_AI_HF_TOKEN
try:
    ensure_models_downloaded(
        qwen_dir=paths.qwen_dir,
        sdxl_dir=paths.sdxl_dir,
        sdxl_inpaint_dir=paths.sdxl_inpaint_dir,
        sam_ckpt=paths.sam_ckpt,
        hf_token=os.environ.get("MANGA_AI_HF_TOKEN"),
    )
    print("Models are present (downloaded if needed).")
except Exception as e:
    raise RuntimeError(
        "Model download/setup failed. If SDXL/Qwen are gated, accept the license on HuggingFace and set MANGA_AI_HF_TOKEN. "
        f"Original error: {e}"
    )


In [ ]:
from pathlib import Path

# Quick sanity checks (doesn't load the full ML pipelines)
assert (paths.qwen_dir / "config.json").exists(), "Qwen config.json not found"
assert (paths.sdxl_dir / "model_index.json").exists(), "SDXL model_index.json not found"
assert (paths.sdxl_inpaint_dir / "model_index.json").exists(), "SDXL inpaint model_index.json not found"
assert paths.sam_ckpt.exists(), "SAM checkpoint not found"

print("Sanity checks passed.")
print("SAM checkpoint size (MB):", round(paths.sam_ckpt.stat().st_size / (1024 * 1024), 1))


## Launch Web UI + Cloudflare Public URL

Running the next cell will:
- Start Gradio locally on port `7860`
- Start a Cloudflare tunnel and print a public `https://...trycloudflare.com` URL


In [ ]:
import os
from manga_ai.scripts.cloudflare_tunnel import start_tunnel
from manga_ai.scripts.gradio_app import launch_gradio

PORT = int(os.environ.get("MANGA_AI_PORT", "7860"))
HOST = os.environ.get("MANGA_AI_HOST", "127.0.0.1")

launch_gradio(root=ROOT, host=HOST, port=PORT)

tunnel = start_tunnel(local_port=PORT, cache_dir=ROOT / "outputs" / "cloudflared", host=HOST)
print("Public URL:", tunnel.public_url)
